## Via original pipeline and Baselines.py

In [4]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- 1. Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features
# --- WICHTIG: Wir importieren ALLE Modelle, die wir brauchen ---
from src.models.Baselines import ForecastModel as BaselineModel
from src.models.EN import ForecastModel as ENModel # Für AR(1)

print('PROJECT_ROOT =', PROJECT_ROOT)

# --- 2. Daten laden ---
y = load_target()  # ΔIP with DatetimeIndex

# --- 3. Base config (splits & policy as in thesis) ---
def base_cfg() -> GlobalConfig:
    cfg = GlobalConfig()
    cfg.W0_A     = 180 #
    cfg.BLOCKS_A = [(181,200), (201,220), (221,240)] #
    cfg.W0_B     = 240 #
    # Policy ist für Baselines irrelevant, da wir nur 1 Config pro Modell haben
    cfg.policy_window   = 12 #
    cfg.policy_decay    = 0.95 #
    cfg.policy_gain_min = 0.03 #
    cfg.policy_cooldown = 3 #
    return cfg

cfg0 = base_cfg()

# --- 4. Helper für Korrelations-Spezifikation ---
def make_corr_spec(kind: str) -> dict:
    spec = dict(DEFAULT_CORR_SPEC) #
    spec['mode'] = kind #
    return spec

# ======================================================================
# --- TEIL 1: AR(1) MODELL (ALS LASSO AUF LAGS) ---
# ======================================================================
print("\n--- Starte Lauf für AR(1) ---")
MODEL_NAME_AR = "baseline_ar1" #
outputs_for_model(MODEL_NAME_AR) #

# Trick: Statt ifo-Features verwenden wir Lags von y als X-Matrix
X_ar = pd.DataFrame(index=y.index) #
X_ar['y_lag1'] = y.shift(1) # Nur Lag 1 für AR(1)
# (Man könnte hier auch mehr Lags (z.B. bis 12) hinzufügen für ein AR(p))

idx_ar = y.index.intersection(X_ar.index) #
y_ar, X_ar_sync = y.loc[idx_ar], X_ar.loc[idx_ar] #

# Grid für AR(1):
# Wir verwenden ElasticNet (Lasso) als AR(1)-Schätzer
# Wir müssen die FE-Pipeline anweisen, die Features NICHT zu verändern.
model_grid_ar = [{
    'alpha': 1.0,               # Reines Lasso (entspricht AR(1)-Selektion)
    'corr_tag': 'expanding', #
    'corr_spec': make_corr_spec('expanding'), #
    'lag_candidates': (0,),     # Nimm das Feature, wie es ist (y_lag1)
    'top_k_lags_per_feature': 1, #
    'use_rm3': False,           # Keine Glättung
    'k1_topk': 1,               # Wähle nur das Top-1 Feature (also y_lag1)
    'redundancy_param': 0.99, #
    'dr_method': 'none',        # Keine DR
    'target_block_set': None,   # Keine externen Blöcke
    'sample_weight_decay': None # Kein Weighting
}]

# --- Stage A/B für AR(1) ---
shortlist_ar = run_stageA(
    model_name=MODEL_NAME_AR, #
    model_ctor=lambda hp: ENModel(hp), #
    model_grid=model_grid_ar, #
    X=X_ar_sync, y=y_ar, cfg=cfg0, #
    keep_top_k_final=1, #
    min_survivors_per_block=1, #
)

run_stageB(
    model_name=MODEL_NAME_AR, #
    model_ctor=lambda hp: ENModel(hp), #
    shortlist=shortlist_ar, #
    X=X_ar_sync, y=y_ar, cfg=cfg0, #
)

# ======================================================================
# --- TEIL 2: RW und Mean MODELLE ---
# ======================================================================

# Wir laden die ifo-Features, nur damit die Pipeline X-Daten hat
# Das BaselineModel wird sie ignorieren
X_ifo = load_ifo_features() #
idx_ifo = y.index.intersection(X_ifo.index) #
y_ifo, X_ifo_sync = y.loc[idx_ifo], X_ifo.loc[idx_ifo] #

# --- Basis-Grid für RW/Mean ---
# Die FE/DR-Parameter sind irrelevant, da X ignoriert wird,
# aber wir müssen sie setzen, damit die Pipeline läuft.
base_grid_config = {
    'corr_tag': 'expanding', #
    'corr_spec': make_corr_spec('expanding'), #
    'lag_candidates': (1,), #
    'top_k_lags_per_feature': 1, #
    'use_rm3': False, #
    'k1_topk': 1, #
    'redundancy_param': 0.99, #
    'dr_method': 'none', #
    'target_block_set': None, #
    'sample_weight_decay': None #
}

# --- 2a) Random Walk (RW) ---
print("\n--- Starte Lauf für Random Walk (RW) ---")
MODEL_NAME_RW = "baseline_rw" #
outputs_for_model(MODEL_NAME_RW) #

model_grid_rw = [{
    **base_grid_config, #
    'model_type': 'RW' #
}]

shortlist_rw = run_stageA(
    model_name=MODEL_NAME_RW, #
    model_ctor=lambda hp: BaselineModel(hp), #
    model_grid=model_grid_rw, #
    X=X_ifo_sync, y=y_ifo, cfg=cfg0, #
    keep_top_k_final=1, #
    min_survivors_per_block=1, #
)

run_stageB(
    model_name=MODEL_NAME_RW, #
    model_ctor=lambda hp: BaselineModel(hp), #
    shortlist=shortlist_rw, #
    X=X_ifo_sync, y=y_ifo, cfg=cfg0, #
)

# --- 2b) Grand Mean ---
print("\n--- Starte Lauf für Grand Mean ---")
MODEL_NAME_MEAN = "baseline_mean" #
outputs_for_model(MODEL_NAME_MEAN) #

model_grid_mean = [{
    **base_grid_config, #
    'model_type': 'Mean' #
}]

shortlist_mean = run_stageA(
    model_name=MODEL_NAME_MEAN, #
    model_ctor=lambda hp: BaselineModel(hp), #
    model_grid=model_grid_mean, #
    X=X_ifo_sync, y=y_ifo, cfg=cfg0, #
    keep_top_k_final=1, #
    min_survivors_per_block=1, #
)

run_stageB(
    model_name=MODEL_NAME_MEAN, #
    model_ctor=lambda hp: BaselineModel(hp), #
    shortlist=shortlist_mean, #
    X=X_ifo_sync, y=y_ifo, cfg=cfg0, #
)

print(f"\nAlle Baseline-Läufe (AR1, RW, Mean) abgeschlossen.") #
print("Vergleiche die 'summary_active.txt' in den jeweiligen output-Ordnern.") #

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code

--- Starte Lauf für AR(1) ---
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=1
  - Config 1/1
    · Month 1/20 processed | running...RMSE=1.5147
    · Month 2/20 processed | running...RMSE=2.2708
    · Month 3/20 processed | running...RMSE=1.9433
    · Month 4/20 processed | running...RMSE=1.6894
    · Month 5/20 processed | running...RMSE=1.6409
    · Month 6/20 processed | running...RMSE=1.5045
    · Month 7/20 processed | running...RMSE=1.3947
    · Month 8/20 processed | running...RMSE=1.3122
    · Month 9/20 processed | running...RMSE=1.3499
    · Month 10/20 processed | running...RMSE=1.3092
    · Month 11/20 processed | running...RMSE=1.2538
    · Month 12/20 processed | running...RMSE=1.2199
    · Month 13/20 processed | running...RMSE=1.1720
    · Month 14/20 processed | running...RMSE=1.1676
    · Month 15/20 processed | running...RMSE=1.2296
    · Month 16/20 processed | running...RMSE=1.1910
    · Mo

## Direct Implementation

In [6]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error

# --- 1. Konfiguration ---
# n (Start-Trainingsfenster), wie gewünscht auf 240 gesetzt
INITIAL_TRAIN_WINDOW = 240

# --- 2. Pfad-Setup (Stellt sicher, dass 'src' gefunden wird) ---
def _locate_repo_root(start: Path) -> Path:
    """
    Try to locate repo root that contains `src/`
    """
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"PROJECT_ROOT = {PROJECT_ROOT}")

# --- 3. Daten laden ---
try:
    # Wir laden die Target-Serie (IP_change)
    from src.io_timesplits import load_target, load_chronos
    from src.config import PROCESSED
    from statsmodels.tsa.ar_model import AutoReg
except ImportError:
    print("Fehler: 'src'-Module oder 'statsmodels' nicht gefunden.")
    print("Bitte 'statsmodels' installieren: pip install statsmodels")
    sys.exit(1)

# Lade y = IP_change
y = load_target()
# Lade die Chronos-Prognosen (chronos_bolt.parquet)
try:
    df_chronos = load_chronos()
    chronos_mean_preds = df_chronos["chronos_mean"]
    print("Target und Chronos-Prognosen geladen.")
except FileNotFoundError:
    print("Fehler: 'chronos_bolt.parquet' nicht gefunden.")
    print("Bitte das Chronos-Notebook zuerst ausführen.")
    chronos_mean_preds = pd.Series(dtype=float)
except Exception as e:
    print(f"Fehler beim Laden von Chronos: {e}")
    print("Stellen Sie sicher, dass 'io_timesplits.py' die Parquet-Datei korrekt lädt.")
    chronos_mean_preds = pd.Series(dtype=float)


# --- 4. Expanding Window Prognose (Baselines) ---
print(f"Starte Expanding-Window-Prognose für Baselines...")
print(f"Initiales Trainingsfenster n = {INITIAL_TRAIN_WINDOW} Monate.")

T = len(y)
n = INITIAL_TRAIN_WINDOW

# Listen, um die Prognosen zu sammeln
preds_ar1 = []
preds_mean = []
preds_rw = []
# Das erste OOS-Datum ist bei Index n
eval_index = y.iloc[n:].index

# Wir iterieren von t=n bis zum Ende
# t=n: Train[0:n] (Größe n), Predict[n]
# t=n+1: Train[0:n+1] (Größe n+1), Predict[n+1]
for t in tqdm(range(n, T)):
    # y_train enthält alle Daten *vor* dem Zeitpunkt t (expandierendes Fenster)
    y_train = y.iloc[0:t]

    # 1. AR(1) Modell
    # 'trend=c' (konstante) ist Standard
    model_ar1 = AutoReg(y_train, lags=1, trend='c', old_names=False).fit()
    pred_ar1 = model_ar1.forecast(steps=1).iloc[0]
    preds_ar1.append(pred_ar1)

    # 2. Grand Mean Modell
    # Prognose ist der Mittelwert des expandierenden Trainingsfensters
    pred_mean = y_train.mean()
    preds_mean.append(pred_mean)

    # 3. Random Walk (RW) Modell
    # Prognose für die *Änderung* (y) ist immer 0
    preds_rw.append(0.0)

print("Prognose-Berechnung abgeschlossen.")

# --- 5. Ergebnisse auswerten (RMSE) ---

def rmse(y_true, y_pred):
    """Berechnet den Root Mean Squared Error."""
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Erstelle ein DataFrame für einen FAIREN Vergleich
# (d.h. nur Zeitpunkte, für die ALLE Modelle eine Prognose haben)
df_eval = pd.DataFrame(index=eval_index)
df_eval['y_true'] = y.iloc[n:]
df_eval['AR1'] = preds_ar1
df_eval['Mean'] = preds_mean
df_eval['RW'] = preds_rw
df_eval['Chronos'] = chronos_mean_preds.reindex(eval_index)

# Entferne alle Zeilen, in denen irgendein Modell (z.B. Chronos)
# aufgrund seines eigenen Warm-ups (NaN) keine Prognose hatte.
df_eval_clean = df_eval.dropna()

print("\n--- RMSE-Ergebnisse (berechnet auf gemeinsamer OOS-Periode) ---")
print(f"Evaluationsfenster: {df_eval_clean.index[0].date()} bis {df_eval_clean.index[-1].date()}")
print(f"Anzahl der Prognosen: {len(df_eval_clean)} Monate")
print("-" * 60)

# Berechne RMSE für jeden
rmse_ar1 = rmse(df_eval_clean['y_true'], df_eval_clean['AR1'])
rmse_mean = rmse(df_eval_clean['y_true'], df_eval_clean['Mean'])
rmse_rw = rmse(df_eval_clean['y_true'], df_eval_clean['RW'])
rmse_chronos = rmse(df_eval_clean['y_true'], df_eval_clean['Chronos'])

print(f"Random Walk (RW) [Baseline]: {rmse_rw:.6f}")
print(f"AR(1) [Baseline]:             {rmse_ar1:.6f}")
print(f"Grand Mean [Baseline]:        {rmse_mean:.6f}")
print(f"Chronos (Precomputed):      {rmse_chronos:.6f}")
print("-" * 60)

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Target und Chronos-Prognosen geladen.
Starte Expanding-Window-Prognose für Baselines...
Initiales Trainingsfenster n = 240 Monate.


  0%|          | 0/167 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:308: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sta

Prognose-Berechnung abgeschlossen.

--- RMSE-Ergebnisse (berechnet auf gemeinsamer OOS-Periode) ---
Evaluationsfenster: 2011-02-01 bis 2024-11-01
Anzahl der Prognosen: 166 Monate
------------------------------------------------------------
Random Walk (RW) [Baseline]: 2.350417
AR(1) [Baseline]:             2.449661
Grand Mean [Baseline]:        2.354693
Chronos (Precomputed):      2.004886
------------------------------------------------------------
